# Python package for postprocessing the TRUST son files

In [ ]:
from trustutils import run 

run.introduction("Morad Ben Tayeb")
run.description("")
run.TRUST_parameters("1.8.3")

In [ ]:
from trustutils import run 
c1 = run.addCase("sim_1","Cas.data") 
run.printCases()
run.runCases()

## Test case

<figure>
    <center>
    <img src="src/canal.png" width="600" alt="computational_domain">
    <figcaption>Description of the computational domain depicting the probes position
</figcaption>
    </center>
</figure>



## Configuration

This validation file was made to show what can be done with the Python post processing code. In this
test case, we have three folders named sim 1, sim 2, sim 3 with sim 2 the continuation of sim 1 and sim 3
the continuation of sim 2. The goal is to do statistical post processing of the results coming from the
.son and .dt ev files.

## Step time

In this section, we will plot simulation step time. Other quantities can be displayed from .dt ev files as
long as the correct keyword is given. 

<figure>
    <center>
    <img src="src/sim_3/results/fig_1.png" width="500">
    <figcaption>Evolution of step time
</figcaption>
    </center>
</figure>


## Velocity 

Plots the instantaneous probes evolution of velocity field and fluctuations.

<figure>
    <center>
    <img src="src/sim_3/results/fig_2.png" width="500">
    <figcaption>velocity</figcaption>
     <img src="src/sim_3/results/fig_3.png" width="500">
    <figcaption>Velocity fluctuation</figcaption>
    </center>
</figure>


## Density

An example with an other field: the instantaneous density evolution. 

<figure>
    <center>
    <img src="src/sim_3/results/fig_4.png" width="500">
    <figcaption>density instantaneous</figcaption>
    </center>
</figure>


## Pressure

Displaying the pressure evolution, fluctuations and finally spatial mean over the segment.

<figure>
    <center>
    <img src="src/sim_3/results/fig_5.png" width="500">
    <figcaption>Pressure instantaneous</figcaption>
     <img src="src/sim_3/results/fig_6.png" width="500">
    <figcaption>Spatial mean pressure</figcaption>
     <img src="src/sim_3/results/fig_7.png" width="500">
    <figcaption>Spatial mean pressure</figcaption>
    </center>
</figure>

## Average windows on velocity

Example of window averaging

<figure>
    <center>
    <img src="src/sim_3/results/fig_8.png" width="500">
    <figcaption>Average windows</figcaption>
    </center>
</figure>

## Autocorrelation and Spectrum

We can perform an autocorrelation of the velocity signal, the root of parabola fit gives us the taylor micro
scale. After that, the energy spectrum is computed using welch method.

<figure>
    <center>
    <img src="src/sim_3/results/fig_9.png" width="500">
    <figcaption>Autocorrelation and parabola fit</figcaption>
            <img src="src/sim_3/results/fig_10.png" width="500">
    <figcaption>Energy spectrum</figcaption>
    </center>
</figure>


## Taylor Micro Scale

Root of parabola fit of autocorrelation

In [ ]:
from trustutils import plot

data = plot.loadText('./sim_3/script_postprocess.py_Roots_1.txt')
a=plot.Table(["Valeur"])
a.addLigne([data],"Taylor micro scale")
a.df

## Computer performance

In [ ]:
run.tablePerf()

## Python

In [ ]:
run.dumpData("script_postprocess.py")